In [1]:
!pip install deeppavlov

     |████████████████████████████████| 686kB 5.1MB/s 
     |████████████████████████████████| 12.2MB 43.5MB/s 
     |████████████████████████████████| 307kB 44.4MB/s 
     |████████████████████████████████| 61kB 22.2MB/s 
     |████████████████████████████████| 3.4MB 20.9MB/s 
     |████████████████████████████████| 4.1MB 43.8MB/s 
     |████████████████████████████████| 31.2MB 50.0MB/s 
     |████████████████████████████████| 51kB 20.9MB/s 
     |████████████████████████████████| 11.8MB 226kB/s 
     |████████████████████████████████| 12.4MB 40.5MB/s 
     |████████████████████████████████| 61kB 20.3MB/s 
     |████████████████████████████████| 92kB 26.5MB/s 
     |████████████████████████████████| 92kB 28.0MB/s 
     |████████████████████████████████| 51kB 20.8MB/s 
     |████████████████████████████████| 8.0MB 38.5MB/s 
     |████████████████████████████████| 51kB 20.6MB/s 
     |████████████████████████████████| 2.3MB 35.1MB/s 
     |████████████████████████████████| 7.1MB 38.3MB/

In [0]:
import deeppavlov

In [3]:
from deeppavlov.dataset_readers.dstc2_reader import DSTC2DatasetReader

data = DSTC2DatasetReader().read('../../download/dstc2/')

2019-07-02 23:14:08.424 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 95: [downloading data from http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz to ../../download/dstc2/]
2019-07-02 23:14:09.288 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz to ../../download/dstc2/dstc2_v2.tar.gz
100%|██████████| 506k/506k [00:00<00:00, 550kB/s]
2019-07-02 23:14:10.214 INFO in 'deeppavlov.core.data.utils'['utils'] at line 201: Extracting ../../download/dstc2/dstc2_v2.tar.gz archive into ../../download/dstc2
2019-07-02 23:14:10.268 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from ../../download/dstc2/dstc2-trn.jsonlist]
2019-07-02 23:14:10.574 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from ../../download/dstc2/dstc2-val.jsonlist]
2019-07-02 23:14:10.840 INFO in 'deeppavlov.dataset_readers

In [4]:
data['train'][1]

({'intents': [{'act': 'inform', 'slots': [['pricerange', 'cheap']]}],
  'text': 'cheap restaurant'},
 {'act': 'request_food', 'text': 'What kind of food would you like?'})

In [0]:
from overrides import overrides

from deeppavlov.core.common.registry import register
from deeppavlov.core.data.data_learning_iterator import DataLearningIterator


@register('my_dialog_iterator')
class MyDialogDatasetIterator(DataLearningIterator):

    @staticmethod
    def _dialogs(data):
        dialogs = []
        prev_resp_act = None
        for x, y in data:
            if x.get('episode_done'):
                del x['episode_done']
                prev_resp_act = None
                dialogs.append(([], []))
            x['prev_resp_act'] = prev_resp_act
            prev_resp_act = y['act']
            dialogs[-1][0].append(x)
            dialogs[-1][1].append(y)
        return dialogs

    @overrides
    def split(self, *args, **kwargs):
        self.train = self._dialogs(self.train)
        self.valid = self._dialogs(self.valid)
        self.test = self._dialogs(self.test)

In [0]:
iterator = MyDialogDatasetIterator(data)

In [7]:
x_dialog, y_dialog = iterator.train[0]
x_dialog

[{'intents': [], 'prev_resp_act': None, 'text': ''},
 {'intents': [{'act': 'inform', 'slots': [['pricerange', 'cheap']]}],
  'prev_resp_act': 'welcomemsg',
  'text': 'cheap restaurant'},
 {'intents': [{'act': 'inform', 'slots': [['this', 'dontcare']]}],
  'prev_resp_act': 'request_food',
  'text': 'any'},
 {'intents': [{'act': 'inform', 'slots': [['area', 'south']]}],
  'prev_resp_act': 'request_area',
  'text': 'south'},
 {'db_result': {'addr': 'cambridge leisure park clifton way cherry hinton',
   'area': 'south',
   'food': 'chinese',
   'name': 'the lucky star',
   'phone': '01223 244277',
   'postcode': 'c.b 1, 7 d.y',
   'pricerange': 'cheap'},
  'intents': [{'act': 'inform', 'slots': [['area', 'south']]}],
  'prev_resp_act': 'api_call',
  'text': 'south'},
 {'intents': [{'act': 'request', 'slots': [['slot', 'addr']]}],
  'prev_resp_act': 'inform_area+inform_food+offer_name',
  'text': 'address'},
 {'intents': [{'act': 'request', 'slots': [['slot', 'phone']]}],
  'prev_resp_act':

In [8]:
x_dialog[1]

{'intents': [{'act': 'inform', 'slots': [['pricerange', 'cheap']]}],
 'prev_resp_act': 'welcomemsg',
 'text': 'cheap restaurant'}

In [9]:
y_dialog[1]

{'act': 'request_food', 'text': 'What kind of food would you like?'}

## Training bot without NER (named entity recognition)

In [0]:
tokenizer = {
    "class_name": "deeppavlov.models.go_bot.wrapper:DialogComponentWrapper",
    "component": { "class_name": "split_tokenizer" },
    "in": ["x"],
    "out": ["x_tokens"]
}
token_vocabulary = {
    "id": "token_vocab",
    "class_name": "simple_vocab",
    "fit_on": ["x_tokens"],
    "save_path": "{MODELS_PATH}/my_gobot/token.dict",
    "load_path": "{MODELS_PATH}/my_gobot/token.dict"
}
network = {
    "in": ["x"],
    "in_y": ["y"],
    "out": ["y_predicted"],
    "main": True,
    "class_name": "go_bot",
    "load_path": "{MODELS_PATH}/my_gobot/model",
    "save_path": "{MODELS_PATH}/my_gobot/model",
    "debug": False,
    "word_vocab": "#token_vocab",
    "template_path": "{DOWNLOADS_PATH}/dstc2/dstc2-templates.txt",
    "template_type": "DualTemplate",
    "api_call_action": "api_call",
    "use_action_mask": False,
    "network_parameters": {
      "learning_rate": 0.005,
      "dropout_rate": 0.5,
      "l2_reg_coef": 7e-4,
      "hidden_size": 128,
      "dense_size": 160
    },
    "slot_filler": None,
    "intent_classifier": None,
    "embedder": None,
    "bow_embedder": {
      "class_name": "bow",
      "depth": "#token_vocab.__len__()",
      "with_counts": True
    },
    "tokenizer": {
      "class_name": "stream_spacy_tokenizer",
      "lowercase": False
    },
    "tracker": {
      "class_name": "featurized_tracker",
      "slot_names": []
    }
}


basic_config = {
  "dataset_reader": {
    "class_name": "dstc2_reader",
    "data_path": "{DOWNLOADS_PATH}/dstc2"
  },
  "dataset_iterator": {
    "class_name": "my_dialog_iterator"
  },
  "chainer": {
    "in": ["x"],
    "in_y": ["y"],
    "out": ["y_predicted"],
    "pipe": [
        tokenizer,
        token_vocabulary,
        network
    ]
  },
  "train": {
    "epochs": 200,
    "batch_size": 4,

    "metrics": ["per_item_dialog_accuracy"],
    "validation_patience": 10,
    
    "val_every_n_batches": 15,
    "val_every_n_epochs": -1,

    "log_every_n_batches": 15,
    "log_every_n_epochs": -1,
    "show_examples": False,
    "validate_best": True,
    "test_best": True
  },
  "metadata": {
    "variables": {
      "ROOT_PATH": "~/.deeppavlov",
      "DOWNLOADS_PATH": "{ROOT_PATH}/downloads",
      "MODELS_PATH": "./models",
      "CONFIGS_PATH": "./configs"
    },
    "requirements": [
      "{DEEPPAVLOV_PATH}/requirements/tf.txt",
      "{DEEPPAVLOV_PATH}/requirements/fasttext.txt",
      "{DEEPPAVLOV_PATH}/requirements/spacy.txt",
      "{DEEPPAVLOV_PATH}/requirements/en_core_web_sm.txt"
    ],
    "labels": {
      "telegram_utils": "GoalOrientedBot",
      "server_utils": "GoalOrientedBot"
    },
    "download": [
      {
        "url": "http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz",
        "subdir": "{DOWNLOADS_PATH}/dstc2"
      }
    ]
  }
}

In [0]:
from deeppavlov import train_model, build_model

from copy import copy

In [0]:
bot = train_model(basic_config, download=True)

2018-11-28 16:44:45.590 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 205: Starting new HTTP connection (1): files.deeppavlov.ai:80
2018-11-28 16:44:45.631 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 393: http://files.deeppavlov.ai:80 "GET /datasets/dstc2_v2.tar.gz.md5 HTTP/1.1" 200 260
2018-11-28 16:44:45.678 INFO in 'deeppavlov.download'['download'] at line 115: Skipped http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz download because of matching hashes
2018-11-28 16:44:45.679 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /home/vimary/.deeppavlov/downloads/dstc2/dstc2-trn.jsonlist]
2018-11-28 16:44:45.906 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /home/vimary/.deeppavlov/downloads/dstc2/dstc2-val.jsonlist]
2018-11-28 16:44:46.56 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /home/vimar

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.0042}, "time_spent": "0:00:09", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 10}}
{"train": {"epochs_done": 0, "batches_seen": 15, "examples_seen": 60, "metrics": {"per_item_dialog_accuracy": 0.0898}, "time_spent": "0:00:10"}}


2018-11-28 16:45:10.688 INFO in 'deeppavlov.core.commands.train'['train'] at line 459: New best per_item_dialog_accuracy of 0.1436
2018-11-28 16:45:10.688 INFO in 'deeppavlov.core.commands.train'['train'] at line 461: Saving model
2018-11-28 16:45:10.689 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [saving model to /home/vimary/ipavlov/Pilot/examples/tutorials/models/my_gobot/model]
2018-11-28 16:45:10.749 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 354: [saving parameters to /home/vimary/ipavlov/Pilot/examples/tutorials/models/my_gobot/model.json]


{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.1436}, "time_spent": "0:00:18", "epochs_done": 0, "batches_seen": 15, "train_examples_seen": 60, "impatience": 0, "patience_limit": 10}}
{"train": {"epochs_done": 0, "batches_seen": 30, "examples_seen": 120, "metrics": {"per_item_dialog_accuracy": 0.211}, "time_spent": "0:00:19"}}


2018-11-28 16:45:19.548 INFO in 'deeppavlov.core.commands.train'['train'] at line 459: New best per_item_dialog_accuracy of 0.25
2018-11-28 16:45:19.549 INFO in 'deeppavlov.core.commands.train'['train'] at line 461: Saving model
2018-11-28 16:45:19.549 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [saving model to /home/vimary/ipavlov/Pilot/examples/tutorials/models/my_gobot/model]
2018-11-28 16:45:19.609 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 354: [saving parameters to /home/vimary/ipavlov/Pilot/examples/tutorials/models/my_gobot/model.json]


...

2018-11-28 16:50:40.428 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 108: [loading templates from /home/vimary/.deeppavlov/downloads/dstc2/dstc2-templates.txt]
2018-11-28 16:50:40.429 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 111: 46 templates loaded
2018-11-28 16:50:40.429 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 139: Calculated input size for `GoalOrientedBotNetwork` is 496
2018-11-28 16:50:40.883 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 120: [initializing `GoalOrientedBotNetwork` from saved]
2018-11-28 16:50:40.884 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 360: [loading parameters from /home/vimary/ipavlov/Pilot/examples/tutorials/models/my_gobot/model.json]
2018-11-28 16:50:40.885 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 43: [loading model from /home/vimary/ipavlov/Pilot/examples/tutorials/models/my_gobot/model]
2018-11-28 16:50:40.894 INFO in 'tensorflow'['tf_logging'] at line 115: Rest

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.3542}, "time_spent": "0:00:09"}}


2018-11-28 16:50:57.30 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /home/vimary/ipavlov/Pilot/examples/tutorials/models/my_gobot/token.dict]


{"test": {"eval_examples_count": 576, "metrics": {"per_item_dialog_accuracy": 0.3595}, "time_spent": "0:00:09"}}


2018-11-28 16:50:57.273 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 108: [loading templates from /home/vimary/.deeppavlov/downloads/dstc2/dstc2-templates.txt]
2018-11-28 16:50:57.273 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 111: 46 templates loaded
2018-11-28 16:50:57.274 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 139: Calculated input size for `GoalOrientedBotNetwork` is 496
2018-11-28 16:50:57.878 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 120: [initializing `GoalOrientedBotNetwork` from saved]
2018-11-28 16:50:57.879 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 360: [loading parameters from /home/vimary/ipavlov/Pilot/examples/tutorials/models/my_gobot/model.json]
2018-11-28 16:50:57.880 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 43: [loading model from /home/vimary/ipavlov/Pilot/examples/tutorials/models/my_gobot/model]
2018-11-28 16:50:57.891 INFO in 'tensorflow'['tf_logging'] at line 115: Rest

In [0]:
# or load trained bot from disk
bot = build_model(basic_config)

In [0]:
bot(['hi, i want some cheap food'])

['Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?']

In [0]:
bot(['bye'])

['Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?']

In [0]:
# free memory
del bot

## Adding embeddings

In [0]:
tokenizer

{'class_name': 'deeppavlov.models.go_bot.wrapper:DialogComponentWrapper',
 'component': {'class_name': 'split_tokenizer'},
 'in': ['x'],
 'out': ['x_tokens']}

In [0]:
token_vocabulary_emb = copy(token_vocabulary)

token_vocabulary_emb['save_path'] = '{MODELS_PATH}/my_gobot_emb/token.dict'
token_vocabulary_emb['load_path'] = '{MODELS_PATH}/my_gobot_emb/token.dict'

In [0]:
print(network['embedder'])

None


In [0]:
emb_file = "wiki.en.bin" # эмбеддинги для английского, 8 Gb
#emb_file = "wiki.ru.bin" # эмбеддинги для русского, 8 Gb
#emb_file = "lenta_lower_100.bin" # легкие эмбеддинги для русского, 1 Gb

In [0]:
network_emb = copy(network)
network_emb['embedder'] = {
    "class_name": "fasttext",
    "load_path": "{DOWNLOADS_PATH}/embeddings/" + emb_file
}
network_emb['load_path'] = "{MODELS_PATH}/my_gobot_emb/model"
network_emb['save_path'] = "{MODELS_PATH}/my_gobot_emb/model"

In [0]:
print(basic_config['metadata']['download'])

[{'url': 'http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz', 'subdir': '{DOWNLOADS_PATH}/dstc2'}]


In [0]:
config_emb = copy(basic_config)

config_emb['metadata']['download'].append({
    "url": "http://files.deeppavlov.ai/deeppavlov_data/embeddings/" + emb_file,
    "subdir": "{DOWNLOADS_PATH}/embeddings"
})

In [0]:
###

In [0]:
config_emb['chainer']['pipe'] = [
    tokenizer,
    token_vocabulary_emb,
    network_emb
]

In [0]:
bot_emb = train_model(config_emb, download=True)
# bot_emb = build_model(config_emb)

2018-11-28 16:52:45.690 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 205: Starting new HTTP connection (1): files.deeppavlov.ai:80
2018-11-28 16:52:45.726 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 393: http://files.deeppavlov.ai:80 "GET /datasets/dstc2_v2.tar.gz.md5 HTTP/1.1" 200 260
2018-11-28 16:52:45.737 INFO in 'deeppavlov.download'['download'] at line 115: Skipped http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz download because of matching hashes
2018-11-28 16:52:45.738 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 205: Starting new HTTP connection (1): files.deeppavlov.ai:80
2018-11-28 16:52:45.774 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 393: http://files.deeppavlov.ai:80 "GET /deeppavlov_data/embeddings/wiki.en.bin.md5 HTTP/1.1" 200 46
2018-11-28 16:53:01.910 INFO in 'deeppavlov.download'['download'] at line 115: Skipped http://files.deeppavlov.ai/deeppavlov_data/embeddings/wiki.en.bin download because of mat

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.0029}, "time_spent": "0:00:09", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 10}}
{"train": {"epochs_done": 0, "batches_seen": 15, "examples_seen": 60, "metrics": {"per_item_dialog_accuracy": 0.0857}, "time_spent": "0:00:10"}}


2018-11-28 16:53:27.247 INFO in 'deeppavlov.core.commands.train'['train'] at line 459: New best per_item_dialog_accuracy of 0.094
2018-11-28 16:53:27.247 INFO in 'deeppavlov.core.commands.train'['train'] at line 461: Saving model
2018-11-28 16:53:27.248 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [saving model to /home/vimary/ipavlov/Pilot/examples/tutorials/models/my_gobot_emb/model]
2018-11-28 16:53:27.306 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 354: [saving parameters to /home/vimary/ipavlov/Pilot/examples/tutorials/models/my_gobot_emb/model.json]


{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.094}, "time_spent": "0:00:18", "epochs_done": 0, "batches_seen": 15, "train_examples_seen": 60, "impatience": 0, "patience_limit": 10}}
{"train": {"epochs_done": 0, "batches_seen": 30, "examples_seen": 120, "metrics": {"per_item_dialog_accuracy": 0.147}, "time_spent": "0:00:19"}}


2018-11-28 16:53:36.610 INFO in 'deeppavlov.core.commands.train'['train'] at line 459: New best per_item_dialog_accuracy of 0.248
2018-11-28 16:53:36.610 INFO in 'deeppavlov.core.commands.train'['train'] at line 461: Saving model
2018-11-28 16:53:36.611 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [saving model to /home/vimary/ipavlov/Pilot/examples/tutorials/models/my_gobot_emb/model]
2018-11-28 16:53:36.672 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 354: [saving parameters to /home/vimary/ipavlov/Pilot/examples/tutorials/models/my_gobot_emb/model.json]


...

2018-11-28 17:01:35.148 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 108: [loading templates from /home/vimary/.deeppavlov/downloads/dstc2/dstc2-templates.txt]
2018-11-28 17:01:35.149 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 111: 46 templates loaded
2018-11-28 17:01:35.149 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 139: Calculated input size for `GoalOrientedBotNetwork` is 796
2018-11-28 17:01:35.627 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 120: [initializing `GoalOrientedBotNetwork` from saved]
2018-11-28 17:01:35.628 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 360: [loading parameters from /home/vimary/ipavlov/Pilot/examples/tutorials/models/my_gobot_emb/model.json]
2018-11-28 17:01:35.630 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 43: [loading model from /home/vimary/ipavlov/Pilot/examples/tutorials/models/my_gobot_emb/model]
2018-11-28 17:01:35.638 INFO in 'tensorflow'['tf_logging'] at line 1

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.3863}, "time_spent": "0:00:09"}}


2018-11-28 17:01:53.778 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /home/vimary/ipavlov/Pilot/examples/tutorials/models/my_gobot_emb/token.dict]
2018-11-28 17:01:53.781 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['fasttext_embedder'] at line 52: [loading fastText embeddings from `/home/vimary/.deeppavlov/downloads/embeddings/wiki.en.bin`]


{"test": {"eval_examples_count": 576, "metrics": {"per_item_dialog_accuracy": 0.3957}, "time_spent": "0:00:09"}}


2018-11-28 17:02:14.602 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 108: [loading templates from /home/vimary/.deeppavlov/downloads/dstc2/dstc2-templates.txt]
2018-11-28 17:02:14.603 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 111: 46 templates loaded
2018-11-28 17:02:14.603 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 139: Calculated input size for `GoalOrientedBotNetwork` is 796
2018-11-28 17:02:15.73 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 120: [initializing `GoalOrientedBotNetwork` from saved]
2018-11-28 17:02:15.74 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 360: [loading parameters from /home/vimary/ipavlov/Pilot/examples/tutorials/models/my_gobot_emb/model.json]
2018-11-28 17:02:15.76 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 43: [loading model from /home/vimary/ipavlov/Pilot/examples/tutorials/models/my_gobot_emb/model]
2018-11-28 17:02:15.84 INFO in 'tensorflow'['tf_logging'] at line 115: 

In [0]:
bot_emb(['hi, i want cheaply priced food, please'])

['Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?']

In [0]:
bot_emb(['bye'])

['Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?']

In [0]:
# free memory
del bot_emb

## Adding pretrained NER model

In [0]:
tokenizer

{'class_name': 'deeppavlov.models.go_bot.wrapper:DialogComponentWrapper',
 'component': {'class_name': 'split_tokenizer'},
 'in': ['x'],
 'out': ['x_tokens']}

In [0]:
token_vocabulary_ner = copy(token_vocabulary_emb)

token_vocabulary_ner['save_path'] = '{MODELS_PATH}/my_gobot_ner/token.dict'
token_vocabulary_ner['load_path'] = '{MODELS_PATH}/my_gobot_ner/token.dict'

In [0]:
print(network_emb['slot_filler'])

None


##### Посмотрим на предполагаемые входные и выходные данные от NER

In [0]:
ner_model = build_model(deeppavlov.configs.ner.slotfill_dstc2, download=True)

2018-11-28 17:02:15.563 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 205: Starting new HTTP connection (1): files.deeppavlov.ai:80
2018-11-28 17:02:15.618 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 393: http://files.deeppavlov.ai:80 "GET /deeppavlov_data/slotfill_dstc2.tar.gz.md5 HTTP/1.1" 200 442
2018-11-28 17:02:15.630 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 205: Starting new HTTP connection (1): files.deeppavlov.ai:80
2018-11-28 17:02:15.663 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 393: http://files.deeppavlov.ai:80 "GET /deeppavlov_data/slotfill_dstc2.tar.gz HTTP/1.1" 200 640674
2018-11-28 17:02:15.667 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/deeppavlov_data/slotfill_dstc2.tar.gz to /home/vimary/.deeppavlov/slotfill_dstc2.tar.gz
100%|██████████| 641k/641k [00:00<00:00, 16.7MB/s]
2018-11-28 17:02:15.713 INFO in 'deeppavlov.core.data.utils'['utils'] at l

In [0]:
ner_model(['i want cheap food in chinese reastaurant in the south of town'])

[{'pricerange': 'cheap', 'food': 'chinese', 'area': 'south'}]

In [0]:
# free memory
del ner_model

##### Let's add NER into a NLU module of our hybrid bot

In [0]:
network_ner = copy(network_emb)

network_ner['slot_filler'] = {
    'config_path': '{DEEPPAVLOV_PATH}/configs/ner/slotfill_dstc2.json'
}
network_ner['load_path'] = "{MODELS_PATH}/my_gobot_ner/model"
network_ner['save_path'] = "{MODELS_PATH}/my_gobot_ner/model"

##### Let's initialize all possible slot types and add a dialog state tracker

In [0]:
slot_types = ["pricerange", "this", "area", "food", "name"]

In [0]:
print(network_ner['tracker'])

{'class_name': 'featurized_tracker', 'slot_names': []}


In [0]:
network_ner["tracker"] = {
    "class_name": "featurized_tracker",
    "slot_names": slot_types
}

In [0]:
##

In [0]:
config_ner = copy(config_emb)

config_ner['chainer']['pipe'] = [
    tokenizer,
    token_vocabulary_ner,
    network_ner
]

In [0]:
from deeppavlov.core.commands.train import train_evaluate_model_from_config

def train_model(config, download=False):
    train_evaluate_model_from_config(config, download=download, recursive=False)
    return build_model(config, load_trained=True)


bot_ner = train_model(config_ner, download=True)
# bot_ner = build_model(config_ner)

2018-11-28 17:03:13.386 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 205: Starting new HTTP connection (1): files.deeppavlov.ai:80
2018-11-28 17:03:13.421 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 393: http://files.deeppavlov.ai:80 "GET /deeppavlov_data/slotfill_dstc2.tar.gz.md5 HTTP/1.1" 200 442
2018-11-28 17:03:13.424 INFO in 'deeppavlov.download'['download'] at line 115: Skipped http://files.deeppavlov.ai/deeppavlov_data/slotfill_dstc2.tar.gz download because of matching hashes
2018-11-28 17:03:13.425 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 205: Starting new HTTP connection (1): files.deeppavlov.ai:80
2018-11-28 17:03:13.460 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 393: http://files.deeppavlov.ai:80 "GET /datasets/dstc2_v2.tar.gz.md5 HTTP/1.1" 200 260
2018-11-28 17:03:13.483 INFO in 'deeppavlov.download'['download'] at line 115: Skipped http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz download because of matc

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.0774}, "time_spent": "0:00:21", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 10}}
{"train": {"epochs_done": 0, "batches_seen": 15, "examples_seen": 60, "metrics": {"per_item_dialog_accuracy": 0.2409}, "time_spent": "0:00:25"}}


2018-11-28 17:04:16.681 INFO in 'deeppavlov.core.commands.train'['train'] at line 459: New best per_item_dialog_accuracy of 0.2897
2018-11-28 17:04:16.681 INFO in 'deeppavlov.core.commands.train'['train'] at line 461: Saving model
2018-11-28 17:04:16.682 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [saving model to /home/vimary/ipavlov/Pilot/examples/tutorials/models/my_gobot_ner/model]
2018-11-28 17:04:16.735 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 354: [saving parameters to /home/vimary/ipavlov/Pilot/examples/tutorials/models/my_gobot_ner/model.json]


{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.2897}, "time_spent": "0:00:45", "epochs_done": 0, "batches_seen": 15, "train_examples_seen": 60, "impatience": 0, "patience_limit": 10}}
{"train": {"epochs_done": 0, "batches_seen": 30, "examples_seen": 120, "metrics": {"per_item_dialog_accuracy": 0.345}, "time_spent": "0:00:48"}}


2018-11-28 17:04:40.310 INFO in 'deeppavlov.core.commands.train'['train'] at line 459: New best per_item_dialog_accuracy of 0.4177
2018-11-28 17:04:40.311 INFO in 'deeppavlov.core.commands.train'['train'] at line 461: Saving model
2018-11-28 17:04:40.311 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [saving model to /home/vimary/ipavlov/Pilot/examples/tutorials/models/my_gobot_ner/model]
2018-11-28 17:04:40.368 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 354: [saving parameters to /home/vimary/ipavlov/Pilot/examples/tutorials/models/my_gobot_ner/model.json]


...

2018-11-28 17:19:02.713 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 43: [loading model from /home/vimary/.deeppavlov/models/slotfill_dstc2/model]
2018-11-28 17:19:02.726 INFO in 'tensorflow'['tf_logging'] at line 115: Restoring parameters from /home/vimary/.deeppavlov/models/slotfill_dstc2/model
2018-11-28 17:19:02.741 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['fasttext_embedder'] at line 52: [loading fastText embeddings from `/home/vimary/.deeppavlov/downloads/embeddings/wiki.en.bin`]
2018-11-28 17:19:08.890 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 108: [loading templates from /home/vimary/.deeppavlov/downloads/dstc2/dstc2-templates.txt]
2018-11-28 17:19:08.890 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 111: 46 templates loaded
2018-11-28 17:19:08.891 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 139: Calculated input size for `GoalOrientedBotNetwork` is 811
2018-11-28 17:19:09.358 INFO in 'deeppavlov.models.go_bot.networ

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.5163}, "time_spent": "0:00:23"}}


2018-11-28 17:19:55.20 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /home/vimary/ipavlov/Pilot/examples/tutorials/models/my_gobot_ner/token.dict]
2018-11-28 17:19:55.25 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /home/vimary/.deeppavlov/models/slotfill_dstc2/word.dict]
2018-11-28 17:19:55.29 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 100: [loading vocabulary from /home/vimary/.deeppavlov/models/slotfill_dstc2/tag.dict]


{"test": {"eval_examples_count": 576, "metrics": {"per_item_dialog_accuracy": 0.5169}, "time_spent": "0:00:23"}}


2018-11-28 17:19:55.848 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 43: [loading model from /home/vimary/.deeppavlov/models/slotfill_dstc2/model]
2018-11-28 17:19:55.861 INFO in 'tensorflow'['tf_logging'] at line 115: Restoring parameters from /home/vimary/.deeppavlov/models/slotfill_dstc2/model
2018-11-28 17:19:55.882 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['fasttext_embedder'] at line 52: [loading fastText embeddings from `/home/vimary/.deeppavlov/downloads/embeddings/wiki.en.bin`]
2018-11-28 17:20:16.731 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 108: [loading templates from /home/vimary/.deeppavlov/downloads/dstc2/dstc2-templates.txt]
2018-11-28 17:20:16.732 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 111: 46 templates loaded
2018-11-28 17:20:16.732 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 139: Calculated input size for `GoalOrientedBotNetwork` is 811
2018-11-28 17:20:17.227 INFO in 'deeppavlov.models.go_bot.networ

In [0]:
bot_ner(['hi, i want some cheap food in the north part of town'])

2018-11-28 17:20:17.352 WARNING in 'deeppavlov.models.go_bot.bot'['bot'] at line 392: No database specified.
2018-11-28 17:20:17.353 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 393: Made api_call with {'pricerange': 'cheap', 'area': 'north'}, got 0 results.


['Sorry there is no #food restaurant in the north of town.']

In [0]:
bot_ner(['thank you'])

2018-11-28 17:20:17.365 WARNING in 'deeppavlov.models.go_bot.bot'['bot'] at line 392: No database specified.
2018-11-28 17:20:17.365 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 393: Made api_call with {'pricerange': 'cheap', 'area': 'north'}, got 0 results.


['You are welcome!']

In [0]:
bot_ner(['bye'])

['You are welcome!']

In [0]:
# free memory
del bot_ner